# BIG DATA ANALYSIS : Web Scraping
---

<img src="images/http.png" />

## 1. 해당 웹사이트 주소에 요청
근본적으로는 파일 다운로드와 동일

In [ ]:
import requests

In [ ]:
url = "https://en.wikipedia.org/wiki/Big_data"
response = requests.get(url)

In [ ]:
response

In [ ]:
# 반환된 HTML, CSS, javascript 텍스트
print(response.text)

In [ ]:
html_data = response.text

## 2. HTML 데이터를 Parasing 하기 위한 라이브러리 다운로드

In [ ]:
import sys
!{sys.executable} -m pip install beautifulsoup4

In [ ]:
from bs4 import BeautifulSoup


soup = BeautifulSoup(html_data, "html.parser")
#전체 HTML 코드에서 h1이라는 태그를 찾는 방법
b = soup.find('h1')
# HTML태그 전체
print(b)
# 그 안에 텍스트
print(b.text)

## 3. 동적 렌더링으로 제공되는 데이터 확보 방법
- 대부분의 최신 웹사이트는 클라이언트 사이드 렌더링으로, 웹 사이트가 렌더링 된 후에, 추가적으로 데이터 요청
- 위키피디아의 경우 페이지가 로드된후 추가적인 업데이트가 없음 (정적 렌더링 / 서버사이드 렌더링)

https://finance.daum.net/quotes/A005930#influential_investors/home

In [ ]:
url = "https://finance.daum.net/quotes/A005930#influential_investors/home"
response = requests.get(url)
html_data = response.text
print(html_data)

In [ ]:
soup = BeautifulSoup(html_data, "html.parser")
a = soup.find('div',attrs={"id":"wrap"})
print(a)
print(a.text)

In [ ]:
#HTML 코드 파싱
soup = BeautifulSoup(html_data, "html.parser")
#우리가 웹브라우저에서 보는 html 코드를 찾을 수가 없음
a = soup.find('a',attrs={"id":"favorite"})
print(a)
print(a.text)

### 개발자 도구를 활용한 동적 요청 주소 파악
- 스크래핑 하고자하는 웹 브라우저에서 우클릭(right click) 후 검사(inspector) 클릭
- network 탭 클릭
- 데이터를 로드할 액션을 웹페이지 상에서 취함(예: 링크 클릭)

In [ ]:
# 개발자 도구를 통해 알아낸 주소!
url = "https://finance.daum.net/api/investor/days?page=2&perPage=30&symbolCode=A005930&pagination=true"

response = requests.get(url)
html_data = response.text
#하지만 접근 거부 ㅜ
print(html_data)

### header를 추가해서, 웹 브라우저인척 하자!!

In [ ]:
url = "https://finance.daum.net/api/investor/days?page=2&perPage=30&symbolCode=A005930&pagination=true"
headers = {
    "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.121 Safari/537.36",
    "referer": "https://finance.daum.net/quotes/A005930"

}
response = requests.get(url,headers=headers)
data = response.text
print(type(data))
print(data)

### string으로 되어있지만, 내부 내용을 보면 json 포맷

In [ ]:
# json 관련 라이브러리 임포트
import json

#json으로 파싱후 dict로 저장
prices = json.loads(data)
print(type(prices))
print(prices)

In [ ]:
# 이제 이런식으로 값에 접근가능
prices['data'][0]

In [ ]:
# 자주 요청을 하면 회사에서 막을 수도 있으니, 일단 저장을 해놓고 쓰자!
with open("result.json", "w") as outfile: 
    json.dump(data,outfile)